- drop duplicates by MLS
- merge columns
- explodes amenities into individual columns
- calculate sqft when range or room dimensions (only) are given

In [86]:
import pandas as pd
import os
import numpy as np
import ast
import re
from pathlib import Path
from collections import Counter


In [87]:
import re

def get_street_address(address):
    if isinstance(address, str):
        # Step 1: Handle missing or invalid addresses
        address = address.strip()
        
        # Step 2: If there is a dash and the part after the dash seems like an apartment number, split
        if "-" in address:
            parts = address.split("-")
            # Check if the part after the dash is likely an apartment number (contains digits)
            if re.search(r'\d', parts[1]):  # If the second part contains digits, treat as apt number
                address = parts[1]  # This assumes the apartment number is the part after the dash
            else:
                address = parts[0]  # If not, we keep the first part (before the dash)

        # Step 3: Remove any content after the opening parenthesis (e.g., neighborhoods)
        address = address.split("(")[0].strip()

        # Step 4: Remove content after "Toronto" (city name) if necessary
        address = address.split("Toronto")[0].strip()

        # Step 5: Convert to lowercase
        return address.lower()
    return ''  # If it's not a valid string, return an empty string

In [88]:
def numeric_price(df, columns_list):
    """$600,000 (str) --> 600000 (float)"""
    df_copy = df.copy()
    
    for column in columns_list:
        # Remove "$" and "," then convert to numeric, invalid parsing will become NaN
        df_copy[column] = df_copy[column].str.replace("$", "", regex=False).str.replace(",", "", regex=False)
        df_copy[column] = pd.to_numeric(df_copy[column], errors='coerce')  # Convert to float, invalid entries become NaN
        
    return df_copy


In [89]:
def get_realtor_house_sigma_sqft(row):
    """For Realtor dfs.
    Remove 'sqft'.
    If Square footage is a range, take the average.
    If it is a single value, leave as is."""

    invalid_values = ["*******************", "******************", "****************", "N/A", "unknown", "not available", "na"]

    # Check if the row contains any of the invalid patterns (case insensitive)
    if isinstance(row, str) and any(invalid_value in row.lower() for invalid_value in invalid_values):
        return None


    # Make sure to treat row as a string and remove 'sqft'
    if isinstance(row, str):
        row = row.replace("sqft", "")\
                    .replace("feet²", "")\
                    .replace("FT", "")\
                    .replace("+", "")\
                    .replace("<", "")\
                    .replace("<", "")\
                    .strip()  # Remove 'units' and descriptors and strip spaces
                    # .replace("*", "")\

        # Check if it's a range (contains '-')
        if "-" in row:
            lower, upper = row.split("-")
            return (float(lower.strip()) + float(upper.strip())) / 2
        
        if "x" in row: # Room dimensions are in meters (later on), but Size always has units of sqft on the website
            length, width = row.split("x")
            return (float(length.strip()) * float(width.strip()))
        
        else:
            return float(row)
    
    else:
        return None

In [90]:
def get_zolo_sqft(row, room_dimensions=None):
    """For Zolo dfs. 
    If Size (sq ft) is NaN, then replaces with the sqft calculated from the room dimensions.
    If Size (sq ft) is a range (i.e., contains "-"), replaces with the average of the bounds."""
    
    # If the 'Size (sq ft)' is a range (contains "-"):
    if isinstance(row, str) and '-' in row:
        lower, upper = row.split("-")
        # Calculate the average of the two bounds
        return (float(lower) + float(upper)) / 2
    
    # If 'Size (sq ft)' is NaN, calculate from room dimensions
    elif pd.isna(row) and room_dimensions:  
        try:
            # Check if 'room_dimensions' is a list-like or string that we can process
            dimensions = ast.literal_eval(room_dimensions)
            square_meters = 0

            # If room dimensions is a list of room sizes (i.e., '200x300', '100x200', etc.)
            if isinstance(dimensions, list):
                for dimension in dimensions:
                    # Only split if 'dimension' contains 'x'
                    length, width = dimension.split("x")
                    square_meters += float(length.strip()) * float(width.strip())
                    
            return square_meters*10.7639
        
        except (ValueError, SyntaxError, TypeError):
            # Handle invalid or malformed room dimensions
            return np.nan

    elif isinstance(row, str):
        # If it's a string that doesn't contain a "-", remove any "+" and return the value as float
        return float(row.replace("+", "").replace(">", "").replace("<", "").strip())
    
    else:
        # If it's a numeric value (already a number), return it
        return row


In [91]:
# Clean the string value associated with the key "size" in each dict the "Room Info" dict for each row.

def clean_size_string(size):
    """For House Sigma dfs"""
    # Remove all non-numeric characters and spaces around the dimensions
    cleaned_dimensions = re.sub(r'[^\d. x]', '', size)
    return cleaned_dimensions

# Calculate the area of each room (where each dict in the "Room Info" list represents a different room)

def get_area(room):
    """House Sigma dfs"""
    # Sometimes the "Room Info" list of dictionaries is an empty list [], or there's no key named "size"
    # for certain rooms in the list.
    if room['size'] is None:
        return 0
    
    # Clean to remove the mï¼‰ special characters after the dimensions for each room
    cleaned_size = clean_size_string(room['size'])
    
    # Use regex to identify form and pattern of dimensions in each dictionary in the list of rooms
    # Provided dimensions are actually in meters.
    match = re.match(r'([\d.]+) x ([\d.]+)', cleaned_size)
    if match:
        length = float(match.group(1))  
        width = float(match.group(2))   
        area_m2 = length * width       
        area_ft2 = area_m2 * 10.7639    
        return area_ft2
    else:
        return 0  
    
# Calculate total sqft (sum of the areas of all rooms in the list of dictionaries)

def get_total_sqft(rooms_list):
    return sum(get_area(room) for room in rooms_list)

In [92]:
def create_columns(list_of_dfs):
    new_dfs = []
    for df in list_of_dfs:
        first_row_empty = df.iloc[0].isna().all()
        
        if first_row_empty:
            df.columns = ['Name', 'Property Info', 'Listing Info', 'Room Info', 'description 1', 'description 2', 'link']
            new_dfs.append(df)
        else:
            new_row = pd.DataFrame([df.columns], columns=df.columns)
            df = pd.concat([new_row, df], ignore_index=True)
            df.columns = ['Name', 'Property Info', 'Listing Info', 'Room Info', 'description 1', 'description 2', 'link']
            new_dfs.append(df)
            
    return new_dfs

In [113]:
def clean_real_estate(new_house_sigma, house_sigma_overview):
    
    """Cleans and processes house_sigma dataframes for historical listings."""
    

    ##### House Sigma #####

    # new_house_sigma["Property Info"] = new_house_sigma["Property Info"].apply(ast.literal_eval)
    # new_house_sigma["Listing Info"] = new_house_sigma["Listing Info"].apply(ast.literal_eval)
    # new_house_sigma["Room Info"] = new_house_sigma["Room Info"].apply(ast.literal_eval)

    new_house_sigma["Property Info"] = new_house_sigma["Property Info"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    
    new_house_sigma["Listing Info"] = new_house_sigma["Listing Info"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
    
    new_house_sigma["Room Info"] = new_house_sigma["Room Info"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )

    property_expanded = pd.json_normalize(new_house_sigma['Property Info'])
    listing_expanded = pd.json_normalize(new_house_sigma['Listing Info'])

    # Concatenate the expanded DataFrames with the original DataFrame, excluding the old columns
    df_expanded = pd.concat([new_house_sigma.drop(columns=['Property Info', 'Listing Info']), property_expanded, listing_expanded], axis=1)
    
    

    # Drop the first instance of each duplicate column (property info and listing info had lots of duplicate keys)
    df_expanded = df_expanded.loc[:, ~df_expanded.columns.duplicated(keep='last')]
    
    df_expanded = df_expanded[~df_expanded["Size:"].str.contains(r"\*|FT", na=False)]
    df_expanded['calculated_sqft'] = df_expanded['Room Info'].apply(get_total_sqft)

    df_expanded["Size:"] = df_expanded["Size:"].apply(get_realtor_house_sigma_sqft)

    df_expanded["Size:"] = df_expanded["Size:"].fillna(0)
    df_expanded["sqft"] = df_expanded.apply(lambda row: row["calculated_sqft"] if row["Size:"] == 0 else row["Size:"], axis=1)
    df_expanded = df_expanded.loc[df_expanded['sqft'] != 0]

    house_sigma = df_expanded

    house_sigma["mls"] = house_sigma["Listing #:"]
    house_sigma["beds"] = house_sigma["Bedrooms:"]
    house_sigma["baths"] = house_sigma["Bathrooms:"]
    house_sigma["Heating Type"] = house_sigma["Heating Type:"]
    house_sigma["Air Conditioning"] = house_sigma["Cooling:"]
    house_sigma["Community"] = house_sigma["Community:"]
    
    # house_sigma["Amenities:"] = house_sigma["Amenities:"].str.strip().replace({
    #     "Outdoor Pool": "Pool", 
    #     "Indoor Pool": "Pool"
    # })
    
    house_sigma["Amenities:"] = house_sigma["Amenities:"].apply(
        lambda x: x.replace("Outdoor Pool", "Pool").replace("Indoor Pool", "Pool") if isinstance(x, str) else x
    )


    # Additionally, replace NaN values explicitly with None or "No Amenity"
    house_sigma["Amenities:"] = house_sigma["Amenities:"].fillna("No amenity")

    house_sigma_merged = pd.merge(house_sigma_overview, house_sigma, on="link", how="left")
    
    # Filter out listings for rent and convert price to float.
    
    house_sigma_merged = house_sigma_merged[~house_sigma_merged["strikethrough_price"].str.contains("Monthly | Weekly", case=False, na=False)]
    house_sigma_merged = numeric_price(house_sigma_merged, ["strikethrough_price", "sold_price"])
    house_sigma_merged["price"] = house_sigma_merged["strikethrough_price"]
    
    house_sigma_merged = house_sigma_merged.rename(columns={"Property Type:" : "Building Type"})
    
    return house_sigma_merged

In [122]:
def get_top_amenities(house_sigma):
    """Gets bools for top 10 amenities across all three real estate dfs."""
    
    # Concatenate the three dfs
    concatenated = pd.concat([house_sigma], ignore_index=True)

    # Function to handle both strings and lists
    def split_string_or_list(value):
        # If the value is a string and contains commas, split it
        if isinstance(value, str):
            return value.split(',')  # Split by commas to create a list
        elif isinstance(value, list):
            return value  # Return the list as is
        return []  # In case of other types (e.g., NaN or unexpected values)

    # Exploding the amenities columns, one by one, and handling both strings and lists
    # features_exploded = concatenated['Features'].apply(split_string_or_list).explode().dropna()
    # building_amenities_exploded = concatenated['Building Amenities'].apply(split_string_or_list).explode().dropna()
    # amenity_exploded = concatenated['Amenity'].apply(split_string_or_list).explode().dropna()
    amenities_exploded = concatenated['Amenities:'].apply(split_string_or_list).explode().dropna()

    # Concatenate all the exploded lists into a single Series
    all_amenities = pd.concat([amenities_exploded])
    # all_amenities = pd.concat([features_exploded, building_amenities_exploded, amenity_exploded, amenities_exploded])

    # Count the occurrences of each amenity
    counter = Counter(all_amenities)
    
    return counter

In [123]:
def merge_real_estate(house_sigma):
    """Gets bools for top 10 amenities across all three real estate dfs."""
    
    # Concatenate all the DataFrames
    concatenated = pd.concat([house_sigma], ignore_index=True)

    # Get the top amenities from the concatenated DataFrame
    # all_amenities = concatenated['Features'].explode().tolist() \
    #     + concatenated['Building Amenities'].explode().tolist() \
    #     + concatenated['Amenity'].explode().tolist() \
    #     + concatenated['Amenities:'].explode().tolist()

    # Get the top 20 amenities based on the count
    counter = get_top_amenities(house_sigma)
    top_20_amenities = counter.most_common(21) # Used top 21 because "" shows up as a frequent "amenity".
    
    # Extract the names of the top 20 amenities
    top_20_amenity_names = [amenity[0].strip() for amenity in top_20_amenities]
    print(top_20_amenity_names)
    # Create a new DataFrame with the boolean columns for each amenity
    amenity_columns = pd.DataFrame()

    for amenity in top_20_amenity_names:
        amenity_columns[amenity] = concatenated.apply(lambda row: 1 if amenity in row.values else 0, axis=1)

    # Concatenate the new boolean columns with the original DataFrame
    concatenated = pd.concat([concatenated, amenity_columns], axis=1)

    # Select the relevant columns
    columns_to_select = [
        "price", 
        "address", 
        "mls", 
        "sqft",
        "Community", 
        "beds", 
        "baths",
        "Building Type",
        "Air Conditioning", 
        "Heating Type",
        "lat",
        "long"
    ] + top_20_amenity_names  

    selected_data = concatenated[columns_to_select]
    
    return selected_data


In [116]:
def final_cleaning(merged):
    """Remove any remaining NaN values by price, address, MLS, and sqft.
    Drop duplicates by MLS across the three real estate data sources."""
    
    merged = merged.dropna(subset=['price', 'address', 'mls', 'sqft'])
                    
    merged = merged.drop_duplicates(subset=["mls"])
    
    return merged

In [97]:
# Listings through the years

parent_dir = Path.cwd().parent
hs0 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 0.csv")
hs2 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 2.csv")
hs3 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 3.csv")
hs4 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 4 - Copy.csv") #, encoding="utf-8", on_bad_lines='skip')
hs5 = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"extracted_houses_housesigma_2003_with_properties 5.csv")

hs_overview = pd.read_csv(parent_dir/"2_data_cleaning"/"cleaned_csv"/"housesigma_data_2003_with_coords.csv")

In [98]:
dfs_list = create_columns([hs0, hs2, hs3, hs4, hs5])

In [99]:
hs_total = pd.concat(dfs_list, ignore_index=True)

In [100]:
hs_total.shape

(87995, 7)

In [ ]:
hs_total.columns
hs_total["Property Info"] = hs_total["Property Info"].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
    )
property_expanded = pd.json_normalize(hs_total['Property Info'])

property_expanded[property_expanded["Lot Size:"] == "50 x 154.5 FT"]


,Tax:,Property Type:,Building Age:,Size:,Lot Size:,Parking:,Basement:,Style:,Community:,Municipality:,...,5 Piece Bathrooms:,5 Piece Ensuite Bathrooms:,4 Piece Ensuite Bathrooms:,Electric:,3 Piece Ensuite Bathrooms:,6 Piece Ensuite Bathrooms:,2 Piece Ensuite Bathrooms:,Ownership Type:,Air Conditioning:,WaterMeter:


In [124]:
house_sigma_cleaned = clean_real_estate(hs_total, hs_overview)
merged = merge_real_estate(house_sigma_cleaned)


['No amenity', 'Bbqs Allowed', 'Visitor Parking', 'Party/Meeting Room', 'Bike Storage', 'Rooftop Deck/Garden', 'Pool', 'Security System', 'Gym', 'Exercise Room', 'Security Guard', 'Visitor Parking', 'Sauna', 'Concierge', 'Recreation Room', 'Guest Suites', 'Concierge', 'Car Wash', 'Exercise Room', 'Bike Storage', 'Pool']


In [126]:
merged = merged.loc[:, ~merged.columns.duplicated(keep='last')]
merged[merged["Pool"] ==1]

,price,address,mls,sqft,Community,beds,baths,Building Type,Air Conditioning,Heating Type,...,Security Guard,Visitor Parking,Sauna,Recreation Room,Guest Suites,Concierge,Car Wash,Exercise Room,Bike Storage,Pool
253,619000.0,"94 - 91 Muir Dr , Scarborough - Scarborough Vi...",E9514728,1299.500000,Scarborough Village,3,2,Condo Townhouse,NaN,Baseboard,...,0,0,0,0,0,0,0,0,0,1
2211,840000.0,"14 Crab Apple Way , North York - Parkwoods-Don...",C9363701,1099.500000,Parkwoods-Donalda,3,2,Condo Townhouse,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,1
4259,1369000.0,"115 - 105 Scenic Mill Way , North York - St. A...",C8475682,1499.500000,St. Andrew-Windfields,3,3,Condo Townhouse,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,1
17168,599000.0,"112 - 112 Scenic Mill Way Way , North York - S...",C5453632,849.500000,St. Andrew-Windfields,1,1,Condo Townhouse,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,1
22871,679000.0,"12 Torrance Rd , Scarborough - Eglinton East",E5001870,1699.500000,Eglinton East,3,3,Condo Townhouse,NaN,Baseboard,...,0,0,0,0,0,0,0,0,0,1
22872,679000.0,"12 Torrance Rd , Scarborough - Eglinton East",E5001870,1699.500000,Eglinton East,3,3,Condo Townhouse,NaN,Baseboard,...,0,0,0,0,0,0,0,0,0,1
23922,599900.0,"114 Scenic Mill Way , North York - St. Andrew-...",C4958753,949.500000,St. Andrew-Windfields,1,1,Condo Townhouse,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,1
24741,578000.0,"56 Golden Appleway , North York - Parkwoods-Do...",C4952274,1499.500000,Parkwoods-Donalda,3,2,Condo Townhouse,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,1
24742,578000.0,"56 Golden Appleway , North York - Parkwoods-Do...",C4952274,1499.500000,Parkwoods-Donalda,3,2,Condo Townhouse,Central Air,Forced Air,...,0,0,0,0,0,0,0,0,0,1
34201,449900.0,"118 - 25 Sunny Glenway Gfwy , Toronto - Flemin...",C4257177,1299.500000,Flemingdon Park,4,2,Condo Townhouse,Window Unit,Baseboard,...,0,0,0,0,0,0,0,0,0,1


In [127]:
merged.isna().sum()

price                    443
address                    0
mls                     3798
sqft                    3798
Community               3798
beds                    3802
baths                   3800
Building Type           3798
Air Conditioning       17417
Heating Type            3819
lat                       33
long                      33
No amenity                 0
Bbqs Allowed               0
Party/Meeting Room         0
Rooftop Deck/Garden        0
Security System            0
Gym                        0
Security Guard             0
Visitor Parking            0
Sauna                      0
Recreation Room            0
Guest Suites               0
Concierge                  0
Car Wash                   0
Exercise Room              0
Bike Storage               0
Pool                       0
dtype: int64

In [128]:
final_merged = final_cleaning(merged)

In [130]:
final_merged["price"].isna().sum()

np.int64(0)

In [131]:
final_merged.to_csv("house_sigma_time_data_check_features.csv")

In [132]:
final_merged.shape

(67824, 28)

In [133]:
final_merged.isna().sum()

price                     0
address                   0
mls                       0
sqft                      0
Community                 0
beds                      4
baths                     2
Building Type             0
Air Conditioning       9635
Heating Type             15
lat                      25
long                     25
No amenity                0
Bbqs Allowed              0
Party/Meeting Room        0
Rooftop Deck/Garden       0
Security System           0
Gym                       0
Security Guard            0
Visitor Parking           0
Sauna                     0
Recreation Room           0
Guest Suites              0
Concierge                 0
Car Wash                  0
Exercise Room             0
Bike Storage              0
Pool                      0
dtype: int64

In [124]:
parent_dir = Path.cwd().parent
realtor = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"realtorcom_listings.csv")
zolo = pd.read_csv(parent_dir/"1_data_extraction"/"newest_zolo_with_mls.csv")
house_sigma = pd.read_csv(parent_dir/"0_raw_data"/"house_data"/"properties_housesigma.csv")

In [ ]:
# Numeric price
# house_sigma = numeric_price(house_sigma, ["Listed Price", "Sold Price"])

new_row = pd.DataFrame([house_sigma.columns], columns=house_sigma.columns)

# Step 2: Append the new row at the top of the DataFrame
new_house_sigma = pd.concat([new_row, house_sigma], ignore_index=True)

# Step 3: Set new column names (make sure the number of new column names matches the number of columns)
new_house_sigma.columns = ['Name', 'Property Info', 'Listing Info', 'Room Info', 'description 1', 'description 2', 'link']

new_house_sigma["Property Info"] = new_house_sigma["Property Info"].apply(ast.literal_eval)
new_house_sigma["Listing Info"] = new_house_sigma["Listing Info"].apply(ast.literal_eval)
new_house_sigma["Room Info"] = new_house_sigma["Room Info"].apply(ast.literal_eval)

property_expanded = pd.json_normalize(new_house_sigma['Property Info'])
listing_expanded = pd.json_normalize(new_house_sigma['Listing Info'])

# Concatenate the expanded DataFrames with the original DataFrame, excluding the old columns
df_expanded = pd.concat([new_house_sigma.drop(columns=['Property Info', 'Listing Info']), property_expanded, listing_expanded], axis=1)
    
df_expanded['calculated_sqft'] = df_expanded['Room Info'].apply(get_total_sqft)


df_expanded = df_expanded.loc[:, ~df_expanded.columns.duplicated(keep='last')]
size_columns = df_expanded.columns[df_expanded.columns.str.contains("Size:")]
size_columns

df_expanded["Size:"] = df_expanded["Size:"].fillna(0)

df_expanded["sqft"] = df_expanded.apply(lambda row: row["calculated_sqft"] if row["Size:"] == 0 else row["Size:"], axis=1)

house_sigma = df_expanded

house_sigma

,Name,Room Info,description 1,description 2,link,Tax:,Property Type:,Maintenance:,Included Utility:,Exposure:,...,Driveway Parking:,Parking Features:,Frontage Length:,Waterfront Features:,View:,Sloping:,Skiing:,Rolling:,calculated_sqft,sqft
0,"Key facts for Unit 508 - 40 Richview Rd, Humbe...","[{'type': 'Dining', 'size': '(4.70 x 3.22 m）',...","Discover this rare, spacious, and beautifully ...",['Spacious and beautifully renovated corner su...,/on/etobicoke-real-estate/508-40-richview-rd/h...,"$2, 265 / 2024",Condo Apt,$1113/month,"water, hydro, heat",Nw,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,844.716428,1200-1399 feet²
1,"Key facts for 24 Wallis Cres, Mount Olive-Silv...","[{'type': 'Living', 'size': '(5.48 x 3.05 m）',...",Welcome to 24 Wallis Cres. Make this detached ...,['Detached home located at 24 Wallis Cres in a...,/on/etobicoke-real-estate/24-wallis-cres/home/...,"$3, 039 / 2024",Detached,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,825.374776,825.374776
2,"Key facts for Unit 2611 - 8 Eglinton Ave E, Mo...","[{'type': 'Kitchen', 'size': '(7.04 x 3.04 m）'...",Welcome To The Award Winning E-Condos In Highl...,['Award-winning E-Condos located in the desira...,/on/toronto-real-estate/2611-8-eglinton-ave-e/...,"$3, 627 / 2024",Condo Apt,$581/month,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,620.095362,600-699 feet²
3,"Key facts for Unit 405 - 35 Fontenay Crt, Eden...","[{'type': 'Den', 'size': '(2.74 x 2.43 m）', 'l...",Boutique Building! One Bedroom + Den; 2 Bathro...,"[""Luxury living in a boutique building featuri...",/on/etobicoke-real-estate/405-35-fontenay-crt/...,"$2, 700 / 2024",Condo Apt,$758/month,"water, heat",Se,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607.534967,700-799 feet²
4,"Key facts for Unit 12 - 51 Florence St, Little...",[],Brockton Commons is an exclusive collection of...,['Brockton Commons consists of 36 exclusive bo...,/on/toronto-real-estate/12-51-florence-st/home...,"$3, 285 / 2023",Condo Townhouse,$400/month,water,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,900-999 feet²
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,"Key facts for 282 Westlake Ave, Woodbine-Lumsd...","[{'type': 'Living', 'size': '(5.28 x 3.14 m）',...",Grand Solid Brick Home in Westlake Endless Po...,['Beautiful and well-maintained solid brick ho...,/on/toronto-real-estate/282-westlake-ave/home/...,"$4, 078 / 2024",Detached,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1098.966204,1098.966204
2624,"Key facts for 258 Perth Ave, Dovercourt-Wallac...","[{'type': 'Living', 'size': '(3.05 x 3.20 m）',...","A two storey, end unit, attached row house. Ac...",['Two-storey end unit attached row house with ...,/on/toronto-real-estate/258-perth-ave/home/EXr...,"$4, 371 / 2023",Freehold Townhouse,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1338.706243,1338.706243
2625,"Key facts for 93 Mount Olive Dr, Mount Olive-S...",[],Welcome to an incredible opportunity in the vi...,['Located in the vibrant Mount Olive neighborh...,/on/etobicoke-real-estate/93-mount-olive-dr/ho...,"$3, 369 / 2024",Detached,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0
2626,"Key facts for 9 Maxwell Ave, Yonge-Eglinton, T...","[{'type': 'Living', 'size': '(6.43 x 3.51 m）',...",This exquisite side-centre hall home in Toront...,"[""Exquisite side-centre hall home located in T...",/on/toronto-real-estate/9-maxwell-ave/home/VgA...,"$8, 998 / 2024",Detached,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1562.018418,1562.018418
